## load libraries

In [1]:
import pandas as pd
import geopandas as gpd
import folium
import fiona
import rasterio
import dask_geopandas as dg

from shapely.geometry import Polygon, Point
import numpy as np
import xarray as xr
import random
import os
import time
import re
import math
import gc
import psutil

import matplotlib.pyplot as plt
import altair as alt
import dask.dataframe as dd

from osgeo import ogr

In [2]:
# Set GeoPandas to use pyogrio
gpd.options.io_engine = "pyogrio"

In [3]:
gpd.show_versions()


SYSTEM INFO
-----------
python     : 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:13:44) [Clang 16.0.6 ]
executable : /Users/jwhite/miniforge3/envs/siads699b/bin/python
machine    : macOS-14.5-arm64-arm-64bit

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : 3.12.2
GEOS lib   : None
GDAL       : 3.9.1
GDAL data dir: /Users/jwhite/miniforge3/envs/siads699b/share/gdal/
PROJ       : 9.4.0
PROJ data dir: /Users/jwhite/miniforge3/envs/siads699b/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 1.0.1
numpy      : 2.0.0
pandas     : 2.2.2
pyproj     : 3.6.1
shapely    : 2.0.5
pyogrio    : 0.9.0
geoalchemy2: 0.15.2
geopy      : 2.4.1
matplotlib : 3.9.1
mapclassify: 2.6.1
fiona      : 1.9.6
psycopg    : 3.2.1
psycopg2   : 2.9.9 (dt dec pq3 ext lo64)
pyarrow    : 16.1.0


## load crop sequence boundaries

In [4]:
# field_sample_data_file = '../data/agricultural/CSB/siads696/geo_balanced_sample.parquet'
field_sample_data_file = '../data/fields/geo_balanced_sample.parquet'
fields = gpd.read_parquet(field_sample_data_file)
fields

,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,INSIDE_X,INSIDE_Y,Shp_Len,Shp_Area,geometry,Longitude,Latitude,Elevation,color,Crop
478801,081623012787392,1623,3.791540,4,24,61,61,24,24,61,...,-6.035779e+05,1.704774e+06,532.211689,15343.877410,"MULTIPOLYGON (((-603525.799 1704829.066, -6035...",-102.962080,38.176922,1231,#bfbf7a,Fallow/Idle Cropland
107446,351623001627247,1623,4.325446,225,1,225,152,152,152,36,...,-7.967410e+05,1.149389e+06,723.964572,17504.529797,"MULTIPOLYGON (((-796679.747 1149498.895, -7966...",-104.600667,33.089835,1138,#ffa8e3,Alfalfa
121162,351623002777545,1623,3.222010,61,176,1,176,36,36,1,...,-1.086137e+06,1.590736e+06,549.623304,13039.063523,"MULTIPOLYGON (((-1086091.187 1590803.212, -108...",-108.309751,36.728108,1676,#ffd400,Corn
88813,351623000046581,1623,10.204036,36,36,36,36,152,37,37,...,-9.788489e+05,1.595846e+06,1274.741723,41294.433943,"MULTIPOLYGON (((-978851.432 1595726.11, -97884...",-107.112180,36.891083,2252,#a5f58d,Other Hay/Non Alfalfa
516266,081623013377366,1623,2.918481,24,29,29,24,1,61,61,...,-6.203751e+05,1.992249e+06,556.566951,11810.721459,"MULTIPOLYGON (((-620252.778 1992330.141, -6202...",-103.412893,40.723858,1330,#bfbf7a,Fallow/Idle Cropland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119071,351623002568175,1623,2.957066,176,176,176,61,24,24,24,...,-9.138551e+05,1.348095e+06,599.379620,11966.871324,"MULTIPOLYGON (((-913829.788 1348147.68, -91380...",-106.082615,34.756107,1877,#e9ffbe,Grass/Pasture
540278,081623014401456,1623,9.374885,61,24,61,1,1,1,1,...,-6.892023e+05,1.934718e+06,988.353218,37938.965442,"MULTIPOLYGON (((-689103.22 1934811.717, -68911...",-104.172849,40.161427,1417,#a5f58d,Other Hay/Non Alfalfa
591949,041623013556134,1623,3.770585,72,72,72,190,190,190,190,...,-1.707418e+06,1.273220e+06,523.387689,15259.076277,"MULTIPOLYGON (((-1707373.732 1273277.961, -170...",-114.508535,33.028054,101,#80b3b3,Woody Wetlands
217842,081623005639708,1623,4.307316,61,24,1,61,24,29,61,...,-6.004245e+05,1.898598e+06,634.862780,17431.160158,"MULTIPOLYGON (((-600373.153 1898692.428, -6003...",-103.092389,39.903658,1396,#a87000,Winter Wheat


In [5]:
fields.crs

<Projected CRS: {"$schema": "https://proj.org/schemas/v0.7/projjso ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

## load soil boundaries

soil boundaries are in 'SAPOLYGON' layer

In [6]:
# Path to your .gdb folder
gdb_path = "../datasets/soil/gNATSGO_CONUS/gNATSGO_CONUS.gdb"  # Replace with your .gdb folder path

# List the layers available in the .gdb file
layers = fiona.listlayers(gdb_path)
print("Available layers:", layers)

Available layers: ['chaashto', 'chconsistence', 'chdesgnsuffix', 'chfrags', 'chorizon', 'chpores', 'chstruct', 'chstructgrp', 'chtext', 'chtexture', 'chtexturegrp', 'chtexturemod', 'chunified', 'cocanopycover', 'cocropyld', 'codiagfeatures', 'coecoclass', 'coeplants', 'coerosionacc', 'coforprod', 'coforprodo', 'cogeomordesc', 'cohydriccriteria', 'cointerp', 'comonth', 'component', 'copm', 'copmgrp', 'copwindbreak', 'corestrictions', 'cosoilmoist', 'cosoiltemp', 'cosurffrags', 'cosurfmorphgc', 'cosurfmorphhpp', 'cosurfmorphmr', 'cosurfmorphss', 'cotaxfmmin', 'cotaxmoistcl', 'cotext', 'cotreestomng', 'cotxfmother', 'distinterpmd', 'distlegendmd', 'distmd', 'featdesc', 'laoverlap', 'legend', 'legendtext', 'mapunit', 'month', 'muaggatt', 'muaoverlap', 'mucropyld', 'mutext', 'sacatalog', 'sainterp', 'sdvalgorithm', 'sdvattribute', 'sdvfolder', 'sdvfolderattribute', 'mdstatdomdet', 'mdstatdommas', 'mdstatidxdet', 'mdstatidxmas', 'mdstatrshipdet', 'mdstatrshipmas', 'mdstattabcols', 'mdstattab

In [7]:
SAPOLYGON_gdf = gpd.read_file(gdb_path, layer='SAPOLYGON')
SAPOLYGON_gdf

/Users/jwhite/miniforge3/envs/siads699b/lib/python3.12/site-packages/pyogrio/raw.py:196: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


,AREASYMBOL,SPATIALVER,LKEY,Shape_Length,Shape_Area,SOURCE,geometry
0,CO636,3.0,14371,1.485387e+05,3.062635e+08,SSURGO,"MULTIPOLYGON (((-778418.1 1630725.1, -778446.3..."
1,KS145,11.0,10599,1.945713e+05,1.954684e+09,SSURGO,"MULTIPOLYGON (((-262470.1 1706056.6, -262483.9..."
2,CO099,13.0,10655,2.627170e+05,4.258098e+09,SSURGO,"MULTIPOLYGON (((-523488.3 1709604.5, -523532.5..."
3,CO011,12.0,10644,2.549603e+05,3.991455e+09,SSURGO,"MULTIPOLYGON (((-583830.5 1713440.2, -583877.8..."
4,CO089,11.0,10653,2.656766e+05,3.286150e+09,SSURGO,"MULTIPOLYGON (((-649477.6 1718195.3, -647923 1..."
...,...,...,...,...,...,...,...
16587,MN,0.0,87281,1.080000e+03,2.790000e+04,DRSS,"MULTIPOLYGON (((-46185 2497485, -46185 2497425..."
16588,MN,0.0,87281,1.020000e+03,2.700000e+04,DRSS,"MULTIPOLYGON (((-46245 2498205, -46245 2498175..."
16589,MN,0.0,87281,1.800000e+03,4.320000e+04,DRSS,"MULTIPOLYGON (((-45585 2498205, -45585 2498145..."
16590,MN,0.0,87281,9.600000e+02,1.980000e+04,DRSS,"MULTIPOLYGON (((-46095 2498265, -46095 2498235..."


In [8]:
SAPOLYGON_gdf.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [9]:
len(SAPOLYGON_gdf['LKEY'].unique())

3187

## join fields with soils

Only need soil data for the fields in the field dataframe: AZ CO UT NM

In [10]:
# Ensure both GeoDataFrames have the same CRS
if fields.crs != SAPOLYGON_gdf.crs:
    fields = fields.to_crs(SAPOLYGON_gdf.crs)

In [11]:
fields.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [12]:
fields_joined_with_soils = gpd.sjoin(fields, SAPOLYGON_gdf[['LKEY', 'geometry']], how='left', predicate='intersects')

In [13]:
fields_joined_with_soils= fields_joined_with_soils.rename(columns={'index_right': 'index_SAPOLYGON'})
fields_joined_with_soils.columns

Index(['CSBID', 'CSBYEARS', 'CSBACRES', 'CDL2016', 'CDL2017', 'CDL2018',
       'CDL2019', 'CDL2020', 'CDL2021', 'CDL2022', 'CDL2023', 'STATEFIPS',
       'STATEASD', 'ASD', 'CNTY', 'CNTYFIPS', 'INSIDE_X', 'INSIDE_Y',
       'Shp_Len', 'Shp_Area', 'geometry', 'Longitude', 'Latitude', 'Elevation',
       'color', 'Crop', 'index_SAPOLYGON', 'LKEY'],
      dtype='object')

In [14]:
fields_joined_with_soils

,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,Shp_Len,Shp_Area,geometry,Longitude,Latitude,Elevation,color,Crop,index_SAPOLYGON,LKEY
478801,081623012787392,1623,3.791540,4,24,61,61,24,24,61,...,532.211689,15343.877410,"MULTIPOLYGON (((-603525.799 1704829.066, -6035...",-102.962080,38.176922,1231,#bfbf7a,Fallow/Idle Cropland,3,10644
107446,351623001627247,1623,4.325446,225,1,225,152,152,152,36,...,723.964572,17504.529797,"MULTIPOLYGON (((-796679.747 1149498.895, -7966...",-104.600667,33.089835,1138,#ffa8e3,Alfalfa,297,10316
121162,351623002777545,1623,3.222010,61,176,1,176,36,36,1,...,549.623304,13039.063523,"MULTIPOLYGON (((-1086091.187 1590803.212, -108...",-108.309751,36.728108,1676,#ffd400,Corn,73,10306
88813,351623000046581,1623,10.204036,36,36,36,36,152,37,37,...,1274.741723,41294.433943,"MULTIPOLYGON (((-978851.432 1595726.11, -97884...",-107.112180,36.891083,2252,#a5f58d,Other Hay/Non Alfalfa,75,10323
516266,081623013377366,1623,2.918481,24,29,29,24,1,61,61,...,556.566951,11810.721459,"MULTIPOLYGON (((-620252.778 1992330.141, -6202...",-103.412893,40.723858,1330,#bfbf7a,Fallow/Idle Cropland,2678,10650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119071,351623002568175,1623,2.957066,176,176,176,61,24,24,24,...,599.379620,11966.871324,"MULTIPOLYGON (((-913829.788 1348147.68, -91380...",-106.082615,34.756107,1877,#e9ffbe,Grass/Pasture,196,18327
540278,081623014401456,1623,9.374885,61,24,61,1,1,1,1,...,988.353218,37938.965442,"MULTIPOLYGON (((-689103.22 1934811.717, -68911...",-104.172849,40.161427,1417,#a5f58d,Other Hay/Non Alfalfa,2712,10660
591949,041623013556134,1623,3.770585,72,72,72,190,190,190,190,...,523.387689,15259.076277,"MULTIPOLYGON (((-1707373.732 1273277.961, -170...",-114.508535,33.028054,101,#80b3b3,Woody Wetlands,293,10263
217842,081623005639708,1623,4.307316,61,24,1,61,24,29,61,...,634.862780,17431.160158,"MULTIPOLYGON (((-600373.153 1898692.428, -6003...",-103.092389,39.903658,1396,#a87000,Winter Wheat,2726,10657


In [15]:
# Count occurrences of each index in the joined DataFrame
duplicate_count = fields_joined_with_soils.index.value_counts()

# Identify indices that appear more than once
indices_with_multiple_matches = duplicate_count[duplicate_count > 1].index
# print(indices_with_multiple_matches)
indices_with_multiple_matches

Index([ 18358, 191645, 182777, 616977, 238939, 458911,  35809,  49047, 427220,
       331058, 618458],
      dtype='int64')

In [16]:
len(indices_with_multiple_matches)

11

In [17]:
for index in indices_with_multiple_matches:
    print(f"Field index {index} appears {duplicate_count[index]} times in the joined DataFrame.")
    display(fields_joined_with_soils.loc[index])

Field index 18358 appears 2 times in the joined DataFrame.


,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,Shp_Len,Shp_Area,geometry,Longitude,Latitude,Elevation,color,Crop,index_SAPOLYGON,LKEY
18358,491623007882821,1623,2.836301,176,152,152,36,36,36,36,...,591.78257,11478.148842,"MULTIPOLYGON (((-1096003.199 2063659.376, -109...",-109.158002,40.898186,1836,#ffa8e3,Alfalfa,2690,14420
18358,491623007882821,1623,2.836301,176,152,152,36,36,36,36,...,591.78257,11478.148842,"MULTIPOLYGON (((-1096003.199 2063659.376, -109...",-109.158002,40.898186,1836,#ffa8e3,Alfalfa,2662,14446


Field index 191645 appears 2 times in the joined DataFrame.


,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,Shp_Len,Shp_Area,geometry,Longitude,Latitude,Elevation,color,Crop,index_SAPOLYGON,LKEY
191645,081623002453777,1623,2.8134,61,24,24,61,24,61,24,...,547.908512,11385.470615,"MULTIPOLYGON (((-1080773.611 1631312.719, -108...",-108.30839,37.093358,2068,#c7d79e,Shrubland,58,10247
191645,081623002453777,1623,2.8134,61,24,24,61,24,61,24,...,547.908512,11385.470615,"MULTIPOLYGON (((-1080773.611 1631312.719, -108...",-108.30839,37.093358,2068,#c7d79e,Shrubland,36,10246


Field index 182777 appears 2 times in the joined DataFrame.


...

,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,Shp_Len,Shp_Area,geometry,Longitude,Latitude,Elevation,color,Crop,index_SAPOLYGON,LKEY
427220,081623011430139,1623,71.840335,1,1,42,1,43,24,1,...,2305.151966,290728.683249,"MULTIPOLYGON (((-510953.903 1916512.002, -5109...",-102.053299,40.115328,1085,#ffd400,Corn,2737,10694
427220,081623011430139,1623,71.840335,1,1,42,1,43,24,1,...,2305.151966,290728.683249,"MULTIPOLYGON (((-510953.903 1916512.002, -5109...",-102.053299,40.115328,1085,#ffd400,Corn,2725,10658


Field index 331058 appears 2 times in the joined DataFrame.


,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,Shp_Len,Shp_Area,geometry,Longitude,Latitude,Elevation,color,Crop,index_SAPOLYGON,LKEY
331058,081623008614096,1623,15.390239,24,1,61,24,1,24,24,...,1438.414522,62282.336247,"MULTIPOLYGON (((-559812.684 1798674.758, -5597...",-102.53465,39.040125,1357,#ffd400,Corn,2862,10645
331058,081623008614096,1623,15.390239,24,1,61,24,1,24,24,...,1438.414522,62282.336247,"MULTIPOLYGON (((-559812.684 1798674.758, -5597...",-102.53465,39.040125,1357,#ffd400,Corn,2795,10648


Field index 618458 appears 2 times in the joined DataFrame.


,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,Shp_Len,Shp_Area,geometry,Longitude,Latitude,Elevation,color,Crop,index_SAPOLYGON,LKEY
618458,041623015988581,1623,7.390075,152,152,152,152,152,37,37,...,1081.111032,29906.691569,"MULTIPOLYGON (((-1594546.867 1603541.712, -159...",-113.983771,36.121223,405,#ff2626,Cotton,125,10288
618458,041623015988581,1623,7.390075,152,152,152,152,152,37,37,...,1081.111032,29906.691569,"MULTIPOLYGON (((-1594546.867 1603541.712, -159...",-113.983771,36.121223,405,#ff2626,Cotton,90,10290


## done with fields and SAPOLYGON_gdf dataframes: remove

In [18]:
del fields
del SAPOLYGON_gdf
gc.collect()

40

## clean and merge soil data

In [19]:
# List the layers available in the .gdb file
layers = fiona.listlayers(gdb_path)
print("Available layers:", layers)

Available layers: ['chaashto', 'chconsistence', 'chdesgnsuffix', 'chfrags', 'chorizon', 'chpores', 'chstruct', 'chstructgrp', 'chtext', 'chtexture', 'chtexturegrp', 'chtexturemod', 'chunified', 'cocanopycover', 'cocropyld', 'codiagfeatures', 'coecoclass', 'coeplants', 'coerosionacc', 'coforprod', 'coforprodo', 'cogeomordesc', 'cohydriccriteria', 'cointerp', 'comonth', 'component', 'copm', 'copmgrp', 'copwindbreak', 'corestrictions', 'cosoilmoist', 'cosoiltemp', 'cosurffrags', 'cosurfmorphgc', 'cosurfmorphhpp', 'cosurfmorphmr', 'cosurfmorphss', 'cotaxfmmin', 'cotaxmoistcl', 'cotext', 'cotreestomng', 'cotxfmother', 'distinterpmd', 'distlegendmd', 'distmd', 'featdesc', 'laoverlap', 'legend', 'legendtext', 'mapunit', 'month', 'muaggatt', 'muaoverlap', 'mucropyld', 'mutext', 'sacatalog', 'sainterp', 'sdvalgorithm', 'sdvattribute', 'sdvfolder', 'sdvfolderattribute', 'mdstatdomdet', 'mdstatdommas', 'mdstatidxdet', 'mdstatidxmas', 'mdstatrshipdet', 'mdstatrshipmas', 'mdstattabcols', 'mdstattab

In [20]:
# # List of candidate layers to inspect
# candidate_layers = ['chorizon', 'component', 'muaggatt']

# # dataframes = []
# for layer in candidate_layers:
#     print(f"Inspecting layer: {layer}")
#     dataframe_name = f'{layer}_gdf'
#     dataframe_name = gpd.read_file(gdb_path, layer=layer)
#     print(dataframe_name.columns)
#     print(dataframe_name.head(), "\n")
#     # dataframes.append(dataframe_name)

```
Inspecting layer: chorizon
Index(['hzname', 'desgndisc', 'desgnmaster', 'desgnmasterprime', 'desgnvert',
       'hzdept_l', 'hzdept_r', 'hzdept_h', 'hzdepb_l', 'hzdepb_r',
       ...
       'ph2osoluble_l', 'ph2osoluble_r', 'ph2osoluble_h', 'ptotal_l',
       'ptotal_r', 'ptotal_h', 'excavdifcl', 'excavdifms', 'cokey', 'chkey'],
      dtype='object', length=171)
    hzname  desgndisc desgnmaster desgnmasterprime  desgnvert  hzdept_l  \
0        C        NaN           C             None        2.0       NaN   
1        A        NaN           A             None        1.0       NaN   
2  C1...C5        NaN           C             None        2.0       NaN   
3       Ap        NaN           A             None        1.0       NaN   
4       Bt        NaN           B             None        2.0       NaN   

   hzdept_r  hzdept_h  hzdepb_l  hzdepb_r  ...  ph2osoluble_l  ph2osoluble_r  \
0        20       NaN       NaN       152  ...            NaN            NaN   
1         0       NaN       NaN        20  ...            NaN            NaN   
2        20       NaN       NaN       152  ...            NaN            NaN   
3         0       NaN       NaN        20  ...            NaN            NaN   
4        13       NaN       NaN        23  ...            NaN            NaN   

   ph2osoluble_h  ptotal_l  ptotal_r  ptotal_h  excavdifcl  excavdifms  \
0            NaN       NaN       NaN       NaN        None        None   
1            NaN       NaN       NaN       NaN        None        None   
2            NaN       NaN       NaN       NaN        None        None   
3            NaN       NaN       NaN       NaN        None        None   
4            NaN       NaN       NaN       NaN        None        None   

      cokey     chkey  
0  24629637  73203533  
1  24629637  73203532  
2  24630059  73204462  
3  24630059  73204461  
4  24630061  73204464  

[5 rows x 171 columns] 

Inspecting layer: component
Index(['comppct_l', 'comppct_r', 'comppct_h', 'compname', 'compkind',
       'majcompflag', 'otherph', 'localphase', 'slope_l', 'slope_r',
       ...
       'flsoilleachpot', 'flsoirunoffpot', 'fltemik2use', 'fltriumph2use',
       'indraingrp', 'innitrateleachi', 'misoimgmtgrp', 'vasoimgtgrp', 'mukey',
       'cokey'],
      dtype='object', length=109)
   comppct_l  comppct_r  comppct_h                   compname  \
0        NaN       60.0        NaN                 Udorthents   
1        NaN       40.0        NaN                       Pits   
2        NaN       30.0        NaN                    Menahga   
3        NaN       60.0        NaN                    Beaches   
4        NaN       10.0        NaN  Very poorly drained soils   

             compkind majcompflag  otherph localphase  slope_l  slope_r  ...  \
0  Taxon above family         Yes     None       None      2.0     26.0  ...   
1  Miscellaneous area         Yes     None     gravel      2.0     26.0  ...   
2              Series         Yes  LOW PPT       None      1.0      9.0  ...   
3  Miscellaneous area         Yes     None       None      NaN      NaN  ...   
4                None         No      None       None      NaN      NaN  ...   

   flsoilleachpot  flsoirunoffpot  fltemik2use  fltriumph2use indraingrp  \
0            None            None          N/A            N/A       None   
1            None            None          N/A            N/A       None   
2            None            None          N/A            N/A       None   
3            None            None          N/A            N/A       None   
4            None            None          N/A            N/A       None   

   innitrateleachi misoimgmtgrp vasoimgtgrp   mukey     cokey  
0              NaN         None        None  398852  24629854  
1              NaN         None        None  398852  24629855  
2              NaN         None        None  398853  24629637  
3              NaN         None        None  398853  24629638  
4              NaN         None        None  398853  24629639  

[5 rows x 109 columns] 

Inspecting layer: muaggatt
Index(['musym', 'muname', 'mustatus', 'slopegraddcp', 'slopegradwta',
       'brockdepmin', 'wtdepannmin', 'wtdepaprjunmin', 'flodfreqdcd',
       'flodfreqmax', 'pondfreqprs', 'aws025wta', 'aws050wta', 'aws0100wta',
       'aws0150wta', 'drclassdcd', 'drclasswettest', 'hydgrpdcd', 'iccdcd',
       'iccdcdpct', 'niccdcd', 'niccdcdpct', 'engdwobdcd', 'engdwbdcd',
       'engdwbll', 'engdwbml', 'engstafdcd', 'engstafll', 'engstafml',
       'engsldcd', 'engsldcp', 'englrsdcd', 'engcmssdcd', 'engcmssmp',
       'urbrecptdcd', 'urbrecptwta', 'forpehrtdcp', 'hydclprs', 'awmmfpwwta',
       'mukey'],
      dtype='object')
  musym                                             muname mustatus  \
0  1030                    Udorthents-Pits, gravel complex     None   
1  1033                            Beaches-Menahga complex     None   
2  1059                                     Wega silt loam     None   
3  1066            Rock outcrop-Garnes complex, very stony     None   
4  1067  Waupaca-Eutroboralfs complex, 0 to 60 percent ...     None   

   slopegraddcp  slopegradwta  brockdepmin  wtdepannmin  wtdepaprjunmin  \
0          26.0          26.0          NaN          NaN             NaN   
1           NaN           9.0          NaN          NaN             NaN   
2           1.0           1.0          NaN         61.0            61.0   
3           NaN           3.0          0.0         75.0            75.0   
4           1.0          11.7          NaN         15.0            15.0   

  flodfreqdcd flodfreqmax  ...      engsldcp     englrsdcd  engcmssdcd  \
0        None        None  ...     Not rated     Not rated   Not rated   
1        None        None  ...     Not rated     Not rated   Not rated   
2        Rare        Rare  ...  Very limited  Very limited        Fair   
3        None        None  ...     Not rated     Not rated   Not rated   
4  Occasional  Occasional  ...  Very limited  Very limited        Poor   

   engcmssmp   urbrecptdcd urbrecptwta forpehrtdcp hydclprs awmmfpwwta   mukey  
0  Not rated     Not rated         NaN   Not rated        0        NaN  398852  
1  Not rated     Not rated       0.296   Not rated       10      1.000  398853  
2  Not rated   Not limited       0.000      Slight       15      0.999  398854  
3  Not rated     Not rated       0.193   Not rated        5      0.873  398855  
4  Not rated  Very limited       1.000      Slight       60      1.000  398856  

[5 rows x 40 columns] 

```

### pH data are in chorizon layer

In [21]:
# Load a specific layer into a GeoDataFrame

# gdf_soil = gpd.read_file(gdb_path, layer='chorizon')
# gdf_soil.to_pickle('../datasets/soil/chorizon_df.pickle')

gdf_soil = gpd.GeoDataFrame(pd.read_pickle('../datasets/soil/chorizon_df.pickle'))

gdf_soil

,hzname,desgndisc,desgnmaster,desgnmasterprime,desgnvert,hzdept_l,hzdept_r,hzdept_h,hzdepb_l,hzdepb_r,...,ph2osoluble_l,ph2osoluble_r,ph2osoluble_h,ptotal_l,ptotal_r,ptotal_h,excavdifcl,excavdifms,cokey,chkey
0,C,NaN,C,None,2.0,NaN,20,NaN,NaN,152,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,24629637,73203533
1,A,NaN,A,None,1.0,NaN,0,NaN,NaN,20,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,24629637,73203532
2,C1...C5,NaN,C,None,2.0,NaN,20,NaN,NaN,152,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,24630059,73204462
3,Ap,NaN,A,None,1.0,NaN,0,NaN,NaN,20,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,24630059,73204461
4,Bt,NaN,B,None,2.0,NaN,13,NaN,NaN,23,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,24630061,73204464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3563603,2A,2.0,A,None,NaN,118.0,168,200.0,148.0,179,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,3314515:2808111,3314515:11333113
3563604,2Bt,2.0,B,None,NaN,148.0,179,200.0,200.0,200,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,3314515:2808111,3314515:11333112
3563605,Oi,NaN,O,None,NaN,0.0,0,0.0,0.0,30,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,3314515:2808112,3314515:11333115
3563606,Oa,NaN,O,None,NaN,0.0,30,46.0,30.0,91,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,3314515:2808112,3314515:11333116


In [22]:
list(gdf_soil.columns)

['hzname',
 'desgndisc',
 'desgnmaster',
 'desgnmasterprime',
 'desgnvert',
 'hzdept_l',
 'hzdept_r',
 'hzdept_h',
 'hzdepb_l',
 'hzdepb_r',
 'hzdepb_h',
 'hzthk_l',
 'hzthk_r',
 'hzthk_h',
 'fraggt10_l',
 'fraggt10_r',
 'fraggt10_h',
 'frag3to10_l',
 'frag3to10_r',
 'frag3to10_h',
 'sieveno4_l',
 'sieveno4_r',
 'sieveno4_h',
 'sieveno10_l',
 'sieveno10_r',
 'sieveno10_h',
 'sieveno40_l',
 'sieveno40_r',
 'sieveno40_h',
 'sieveno200_l',
 'sieveno200_r',
 'sieveno200_h',
 'sandtotal_l',
 'sandtotal_r',
 'sandtotal_h',
 'sandvc_l',
 'sandvc_r',
 'sandvc_h',
 'sandco_l',
 'sandco_r',
 'sandco_h',
 'sandmed_l',
 'sandmed_r',
 'sandmed_h',
 'sandfine_l',
 'sandfine_r',
 'sandfine_h',
 'sandvf_l',
 'sandvf_r',
 'sandvf_h',
 'silttotal_l',
 'silttotal_r',
 'silttotal_h',
 'siltco_l',
 'siltco_r',
 'siltco_h',
 'siltfine_l',
 'siltfine_r',
 'siltfine_h',
 'claytotal_l',
 'claytotal_r',
 'claytotal_h',
 'claysizedcarb_l',
 'claysizedcarb_r',
 'claysizedcarb_h',
 'om_l',
 'om_r',
 'om_h',
 'dbte

In [23]:
pH_columns_to_keep = [
    # 'hzname',
    # 'desgndisc',
    # 'desgnmaster',
    # 'desgnmasterprime',
    # 'desgnvert',
    # 'hzdept_l',
    # 'hzdept_r',
    # 'hzdept_h',
    # 'hzdepb_l',
    # 'hzdepb_r',
    # 'hzdepb_h',
    # 'hzthk_l',
    # 'hzthk_r',
    # 'hzthk_h',
    # 'fraggt10_l',
    # 'fraggt10_r',
    # 'fraggt10_h',
    # 'frag3to10_l',
    # 'frag3to10_r',
    # 'frag3to10_h',
    # 'sieveno4_l',
    # 'sieveno4_r',
    # 'sieveno4_h',
    # 'sieveno10_l',
    # 'sieveno10_r',
    # 'sieveno10_h',
    # 'sieveno40_l',
    # 'sieveno40_r',
    # 'sieveno40_h',
    # 'sieveno200_l',
    # 'sieveno200_r',
    # 'sieveno200_h',
    # 'sandtotal_l',
    # 'sandtotal_r',
    # 'sandtotal_h',
    # 'sandvc_l',
    # 'sandvc_r',
    # 'sandvc_h',
    # 'sandco_l',
    # 'sandco_r',
    # 'sandco_h',
    # 'sandmed_l',
    # 'sandmed_r',
    # 'sandmed_h',
    # 'sandfine_l',
    # 'sandfine_r',
    # 'sandfine_h',
    # 'sandvf_l',
    # 'sandvf_r',
    # 'sandvf_h',
    # 'silttotal_l',
    # 'silttotal_r',
    # 'silttotal_h',
    # 'siltco_l',
    # 'siltco_r',
    # 'siltco_h',
    # 'siltfine_l',
    # 'siltfine_r',
    # 'siltfine_h',
    # 'claytotal_l',
    # 'claytotal_r',
    # 'claytotal_h',
    # 'claysizedcarb_l',
    # 'claysizedcarb_r',
    # 'claysizedcarb_h',
    # 'om_l',
    # 'om_r',
    # 'om_h',
    # 'dbtenthbar_l',
    # 'dbtenthbar_r',
    # 'dbtenthbar_h',
    # 'dbthirdbar_l',
    # 'dbthirdbar_r',
    # 'dbthirdbar_h',
    # 'dbfifteenbar_l',
    # 'dbfifteenbar_r',
    # 'dbfifteenbar_h',
    # 'dbovendry_l',
    # 'dbovendry_r',
    # 'dbovendry_h',
    # 'partdensity',
    # 'ksat_l',
    # 'ksat_r',
    # 'ksat_h',
    # 'awc_l',
    # 'awc_r',
    # 'awc_h',
    # 'wtenthbar_l',
    # 'wtenthbar_r',
    # 'wtenthbar_h',
    # 'wthirdbar_l',
    # 'wthirdbar_r',
    # 'wthirdbar_h',
    # 'wfifteenbar_l',
    # 'wfifteenbar_r',
    # 'wfifteenbar_h',
    # 'wsatiated_l',
    # 'wsatiated_r',
    # 'wsatiated_h',
    # 'lep_l',
    # 'lep_r',
    # 'lep_h',
    # 'll_l',
    # 'll_r',
    # 'll_h',
    # 'pi_l',
    # 'pi_r',
    # 'pi_h',
    # 'aashind_l',
    # 'aashind_r',
    # 'aashind_h',
    # 'kwfact',
    # 'kffact',
    # 'caco3_l',
    # 'caco3_r',
    # 'caco3_h',
    # 'gypsum_l',
    # 'gypsum_r',
    # 'gypsum_h',
    # 'sar_l',
    # 'sar_r',
    # 'sar_h',
    # 'ec_l',
    # 'ec_r',
    # 'ec_h',
    # 'cec7_l',
    # 'cec7_r',
    # 'cec7_h',
    # 'ecec_l',
    # 'ecec_r',
    # 'ecec_h',
    # 'sumbases_l',
    # 'sumbases_r',
    # 'sumbases_h',
    'ph1to1h2o_l',
    'ph1to1h2o_r',
    'ph1to1h2o_h',
    'ph01mcacl2_l',
    'ph01mcacl2_r',
    'ph01mcacl2_h',
    # 'freeiron_l',
    # 'freeiron_r',
    # 'freeiron_h',
    # 'feoxalate_l',
    # 'feoxalate_r',
    # 'feoxalate_h',
    # 'extracid_l',
    # 'extracid_r',
    # 'extracid_h',
    # 'extral_l',
    # 'extral_r',
    # 'extral_h',
    # 'aloxalate_l',
    # 'aloxalate_r',
    # 'aloxalate_h',
    # 'pbray1_l',
    # 'pbray1_r',
    # 'pbray1_h',
    # 'poxalate_l',
    # 'poxalate_r',
    # 'poxalate_h',
    # 'ph2osoluble_l',
    # 'ph2osoluble_r',
    # 'ph2osoluble_h',
    # 'ptotal_l',
    # 'ptotal_r',
    # 'ptotal_h',
    # 'excavdifcl',
    # 'excavdifms',
    'cokey',
    'chkey'   
]

In [24]:
gdf_soil[pH_columns_to_keep]

,ph1to1h2o_l,ph1to1h2o_r,ph1to1h2o_h,ph01mcacl2_l,ph01mcacl2_r,ph01mcacl2_h,cokey,chkey
0,5.6,6.7,7.8,NaN,NaN,NaN,24629637,73203533
1,4.5,5.5,6.5,NaN,NaN,NaN,24629637,73203532
2,5.6,7.0,8.4,NaN,NaN,NaN,24630059,73204462
3,5.6,6.7,7.8,NaN,NaN,NaN,24630059,73204461
4,6.6,7.2,7.8,NaN,NaN,NaN,24630061,73204464
...,...,...,...,...,...,...,...,...
3563603,5.1,6.0,6.5,NaN,NaN,NaN,3314515:2808111,3314515:11333113
3563604,5.1,6.5,7.3,NaN,NaN,NaN,3314515:2808111,3314515:11333112
3563605,NaN,NaN,NaN,3.5,4.0,4.5,3314515:2808112,3314515:11333115
3563606,NaN,NaN,NaN,3.5,4.0,4.5,3314515:2808112,3314515:11333116


In [25]:
# list(gdf_soil[columns_to_keep]['desgndisc'].unique())
gdf_soil[pH_columns_to_keep]['ph1to1h2o_l'].unique()

array([ 5.6,  4.5,  6.6,  6.1,  7.4,  5.1,  nan,  5.5,  3.5,  3.6,  5.8,
        7.9,  7.5,  6.2,  7.8,  7. ,  6.4,  6.8,  7.6,  6. ,  5.2,  6.5,
        5. ,  6.3,  7.1,  7.2,  8. ,  4.8,  5.4,  4.2,  4.4,  4. ,  4.3,
        4.9,  4.6,  5.3,  4.7,  7.3,  6.9,  8.2,  8.4,  8.6,  7.7,  8.5,
        5.7,  5.9,  3.8,  9.1,  6.7,  3. ,  9. ,  8.8,  8.1,  3.2,  3.7,
        4.1,  3.4,  3.9,  3.3,  2.5,  2.7,  3.1,  2.1,  9.5,  2.6,  2.8,
        2. ,  8.3,  8.7,  8.9,  9.2,  1.8,  9.4, 10. ,  2.9,  1. ,  2.2,
        9.3,  1.9,  9.6,  9.9, 10.2], dtype=float32)

In [26]:
gdf_soil[pH_columns_to_keep]['ph1to1h2o_r'].unique()

array([ 6.7,  5.5,  7. ,  7.2,  7.9,  7.5,  5.9,  5.6,  6.6,  6.1,  8. ,
        nan,  6.5,  7.3,  5.4,  5. ,  6.2,  7.6,  5.8,  6. ,  6.3,  8.2,
        7.4,  7.8,  8.3,  6.4,  5.3,  4.6,  4.8,  4.3,  4.5,  4. ,  5.1,
        5.2,  5.7,  4.1,  6.9,  6.8,  8.4,  7.7,  7.1,  8.1,  4.4,  4.7,
        4.9,  8.5,  8.6,  4.2,  8.7,  9. ,  8.8,  9.1,  9.2,  9.3,  8.9,
       10.1,  9.4,  9.6,  3.9,  3.8,  3.6,  9.8, 10. ,  9.5,  3.5,  3.3,
        3.7,  3.1,  2.8,  3.2, 10.7, 10.8, 10.2,  3.4,  9.7,  2. ,  2.9,
       10.3,  9.9, 10.5,  3. , 10.6,  2.7, 10.4], dtype=float32)

In [27]:
gdf_soil[pH_columns_to_keep]['ph1to1h2o_h'].unique()

array([ 7.8,  6.5,  8.4,  7.3,  6. ,  nan,  5.5,  5. ,  4.4,  4.5,  7. ,
        9. ,  8.2,  6.8,  7.4,  7.2,  8. ,  8.5,  7.6,  7.5,  5.9,  6.6,
        7.7,  6.2,  5.8,  5.6,  6.4,  6.3,  6.1,  5.7,  6.7,  7.1,  5.4,
        6.9,  8.6,  8.3,  8.8,  8.1,  9.6, 10. ,  9.4, 11. ,  7.9,  8.7,
        4.7,  5.2,  5.3,  4.8,  8.9,  9.5,  9.3,  9.2,  4.6,  4.9,  5.1,
        4.1,  4.2,  9.1, 10.5,  4. ,  3.8,  3.6,  9.9,  9.8,  9.7, 10.1,
        3.4, 10.4, 10.2,  4.3, 10.8, 10.3,  3.5,  3.7,  3.2, 10.7, 10.6],
      dtype=float32)

In [28]:
gdf_soil[pH_columns_to_keep]['ph01mcacl2_l'].unique()

array([nan, 4.5, 5.6, 5.1, 3.5, 3. , 6.6, 3.6, 6.1, 5.4, 4. , 6. , 5.3,
       5. , 4.7, 4.4, 4.2, 4.6, 5.5, 7.4, 7.2, 7.9, 7.1, 6.3, 7. , 6.5,
       7.5, 6.9, 7.3, 7.7, 8.5, 6.2, 6.8, 6.4, 7.8, 8. , 3.7, 4.1, 8.6,
       8.3, 5.7, 2. , 7.6, 2.5, 6.7, 8.2, 4.9, 3.8, 2.9, 2.7, 3.4, 3.1,
       4.8, 8.4, 3.2, 3.3, 3.9, 4.3, 5.2, 5.8, 8.1, 5.9, 1.8, 9. , 8.7,
       2.8, 1.5, 2.3, 2.4, 1.9, 2.6], dtype=float32)

In [29]:
gdf_soil[pH_columns_to_keep]['ph01mcacl2_r'].unique()

array([ nan,  6.2,  7. ,  6.7,  6.1,  4.6,  5.1,  5.4,  5. ,  5.7,  5.5,
        6.4,  4. ,  6.5,  7.2,  7.5,  4.3,  5.3,  4.8,  4.5,  6. ,  4.7,
        5.8,  5.2,  4.9,  5.6,  6.3,  7.8,  7.9,  6.8,  7.6,  7.1,  6.9,
        7.4,  8.1,  8.5,  8. ,  7.7,  6.6,  8.2,  7.3,  8.3,  5.9,  8.4,
        8.9,  3.9,  4.1,  4.2,  3.5,  8.8,  8.6,  9. ,  8.7,  3.2,  3. ,
        2. ,  3.1,  4.4,  3.8,  3.7,  3.3,  3.4,  3.6,  2.4,  9.2,  2.8,
        2.7,  2.9,  2.6,  9.1,  9.5,  9.6,  9.4,  9.7, 11. ,  2.3,  2.2,
        1.8,  1. ], dtype=float32)

In [30]:
gdf_soil[pH_columns_to_keep]['ph01mcacl2_h'].unique()

array([ nan,  7.8,  8.4,  7.3,  6. ,  6.5,  4.5,  5. ,  5.5,  6.2,  5.8,
        7.1,  5.9,  6.1,  6.7,  7. ,  9. ,  8.1,  8. ,  7.5,  8.5,  7.9,
        7.6,  8.6, 11. ,  6.3,  7.7,  6.9,  8.2,  8.3,  9.5,  4. ,  4.4,
        4.6,  6.8,  7.2,  6.6,  4.8,  4.9,  9.1,  8.7,  3.7,  5.6,  7.4,
        9.6,  8.8,  5.1,  5.2,  5.4,  5.3,  4.7,  6.4,  4.2,  3.5,  5.7,
        8.9, 10.2,  3.3, 10.5,  3.9,  9.3, 10. ,  3.8,  4.1,  4.3, 10.4,
        9.4,  3.4,  9.9,  3.6], dtype=float32)

In [31]:
# list(gdf_soil.columns)
ph_small = gdf_soil[pH_columns_to_keep]
del gdf_soil
gc.collect()  # Force garbage collection

0

In [32]:
ph_small

,ph1to1h2o_l,ph1to1h2o_r,ph1to1h2o_h,ph01mcacl2_l,ph01mcacl2_r,ph01mcacl2_h,cokey,chkey
0,5.6,6.7,7.8,NaN,NaN,NaN,24629637,73203533
1,4.5,5.5,6.5,NaN,NaN,NaN,24629637,73203532
2,5.6,7.0,8.4,NaN,NaN,NaN,24630059,73204462
3,5.6,6.7,7.8,NaN,NaN,NaN,24630059,73204461
4,6.6,7.2,7.8,NaN,NaN,NaN,24630061,73204464
...,...,...,...,...,...,...,...,...
3563603,5.1,6.0,6.5,NaN,NaN,NaN,3314515:2808111,3314515:11333113
3563604,5.1,6.5,7.3,NaN,NaN,NaN,3314515:2808111,3314515:11333112
3563605,NaN,NaN,NaN,3.5,4.0,4.5,3314515:2808112,3314515:11333115
3563606,NaN,NaN,NaN,3.5,4.0,4.5,3314515:2808112,3314515:11333116


In [33]:
# candidate_layers = [
#     'mapunit',
#     'muaggatt',
#     'muaoverlap',
#     'mucropyld',
#     'mutext',
#     'SAPOLYGON',
# ]
# for layer in candidate_layers:
#     print(f"Inspecting layer: {layer}")
#     dataframe_name = f'{layer}_gdf'
#     dataframe_name = gpd.read_file(gdb_path, layer=layer)
#     print(dataframe_name.columns)
#     print(dataframe_name.head(), "\n")
#     # dataframes.append(dataframe_name)

```
Inspecting layer: mapunit
Index(['musym', 'muname', 'mukind', 'mustatus', 'muacres', 'mapunitlfw_l',
       'mapunitlfw_r', 'mapunitlfw_h', 'mapunitpfa_l', 'mapunitpfa_r',
       'mapunitpfa_h', 'farmlndcl', 'muhelcl', 'muwathelcl', 'muwndhelcl',
       'interpfocus', 'invesintens', 'iacornsr', 'nhiforsoigrp', 'nhspiagr',
       'vtsepticsyscl', 'mucertstat', 'lkey', 'mukey'],
      dtype='object')
  musym                                             muname        mukind  \
0  1030                    Udorthents-Pits, gravel complex       Complex   
1  1033                            Beaches-Menahga complex       Complex   
2  1059                                     Wega silt loam  Consociation   
3  1066            Rock outcrop-Garnes complex, very stony       Complex   
4  1067  Waupaca-Eutroboralfs complex, 0 to 60 percent ...   Association   

  mustatus  muacres  mapunitlfw_l  mapunitlfw_r  mapunitlfw_h  mapunitpfa_l  \
0     None    785.0           NaN           NaN           NaN           NaN   
1     None    598.0           NaN           NaN           NaN           NaN   
2     None   2875.0           NaN           NaN           NaN           NaN   
3     None    733.0           NaN           NaN           NaN           NaN   
4     None   8224.0           NaN           NaN           NaN           NaN   

   mapunitpfa_r  ...  muwndhelcl interpfocus invesintens iacornsr  \
0           NaN  ...        None        None        None      NaN   
1           NaN  ...        None        None        None      NaN   
2           NaN  ...        None        None        None      NaN   
3           NaN  ...        None        None        None      NaN   
4           NaN  ...        None        None        None      NaN   

  nhiforsoigrp nhspiagr vtsepticsyscl  mucertstat   lkey   mukey  
0         None      NaN          None        None  13785  398852  
1         None      NaN          None        None  13785  398853  
2         None      NaN          None        None  13785  398854  
3         None      NaN          None        None  13785  398855  
4         None      NaN          None        None  13785  398856  

[5 rows x 24 columns] 

Inspecting layer: muaggatt
Index(['musym', 'muname', 'mustatus', 'slopegraddcp', 'slopegradwta',
       'brockdepmin', 'wtdepannmin', 'wtdepaprjunmin', 'flodfreqdcd',
       'flodfreqmax', 'pondfreqprs', 'aws025wta', 'aws050wta', 'aws0100wta',
       'aws0150wta', 'drclassdcd', 'drclasswettest', 'hydgrpdcd', 'iccdcd',
       'iccdcdpct', 'niccdcd', 'niccdcdpct', 'engdwobdcd', 'engdwbdcd',
       'engdwbll', 'engdwbml', 'engstafdcd', 'engstafll', 'engstafml',
       'engsldcd', 'engsldcp', 'englrsdcd', 'engcmssdcd', 'engcmssmp',
       'urbrecptdcd', 'urbrecptwta', 'forpehrtdcp', 'hydclprs', 'awmmfpwwta',
       'mukey'],
      dtype='object')
  musym                                             muname mustatus  \
0  1030                    Udorthents-Pits, gravel complex     None   
1  1033                            Beaches-Menahga complex     None   
2  1059                                     Wega silt loam     None   
3  1066            Rock outcrop-Garnes complex, very stony     None   
4  1067  Waupaca-Eutroboralfs complex, 0 to 60 percent ...     None   

   slopegraddcp  slopegradwta  brockdepmin  wtdepannmin  wtdepaprjunmin  \
0          26.0          26.0          NaN          NaN             NaN   
1           NaN           9.0          NaN          NaN             NaN   
2           1.0           1.0          NaN         61.0            61.0   
3           NaN           3.0          0.0         75.0            75.0   
4           1.0          11.7          NaN         15.0            15.0   

  flodfreqdcd flodfreqmax  ...      engsldcp     englrsdcd  engcmssdcd  \
0        None        None  ...     Not rated     Not rated   Not rated   
1        None        None  ...     Not rated     Not rated   Not rated   
2        Rare        Rare  ...  Very limited  Very limited        Fair   
3        None        None  ...     Not rated     Not rated   Not rated   
4  Occasional  Occasional  ...  Very limited  Very limited        Poor   

   engcmssmp   urbrecptdcd urbrecptwta forpehrtdcp hydclprs awmmfpwwta   mukey  
0  Not rated     Not rated         NaN   Not rated        0        NaN  398852  
1  Not rated     Not rated       0.296   Not rated       10      1.000  398853  
2  Not rated   Not limited       0.000      Slight       15      0.999  398854  
3  Not rated     Not rated       0.193   Not rated        5      0.873  398855  
4  Not rated  Very limited       1.000      Slight       60      1.000  398856  

[5 rows x 40 columns] 

Inspecting layer: muaoverlap
Index(['areaovacres', 'lareaovkey', 'mukey', 'muareaovkey'], dtype='object')
   areaovacres lareaovkey   mukey muareaovkey
0        785.0     558291  398852    33496990
1        785.0     558292  398852    33497063
2        598.0     558291  398853    33496991
3        598.0     558292  398853    33497064
4       2875.0     558291  398854    33496992 

Inspecting layer: mucropyld
Index(['cropname', 'yldunits', 'nonirryield_l', 'nonirryield_r',
       'nonirryield_h', 'irryield_l', 'irryield_r', 'irryield_h', 'mukey',
       'mucrpyldkey'],
      dtype='object')
           cropname yldunits  nonirryield_l  nonirryield_r  nonirryield_h  \
0           Pasture      AUM            NaN            2.5            NaN   
1  Grass-legume hay     Tons            NaN            1.0            NaN   
2           Pasture      AUM            NaN            3.5            NaN   
3      Spring wheat       Bu            NaN           35.0            NaN   
4      Winter wheat       Bu            NaN           40.0            NaN   

   irryield_l  irryield_r  irryield_h    mukey mucrpyldkey  
0         NaN         NaN         NaN  2936819    16338058  
1         NaN         NaN         NaN  2936819    16338057  
2         NaN         5.5         NaN  2936821    16338031  
3         NaN         NaN         NaN  2936821    16338032  
4         NaN         NaN         NaN  2936821    16338033   

Inspecting layer: mutext
Index(['recdate', 'mapunittextkind', 'textcat', 'textsubcat', 'text', 'mukey',
       'mutextkey'],
      dtype='object')
                  recdate mapunittextkind   textcat textsubcat text   mukey  \
0  2017/09/22 00:00:00+00      Edit notes  CropProd       None    0  398852   
1  2017/09/22 00:00:00+00      Edit notes  CropProd       None   15  398853   
2  2017/09/22 00:00:00+00      Edit notes  CropProd       None   84  398854   
3  2017/09/22 00:00:00+00      Edit notes  CropProd       None    5  398855   
4  2017/09/22 00:00:00+00      Edit notes  CropProd       None   15  398856   

  mutextkey  
0   2741010  
1   2741011  
2   2741012  
3   2741013  
4   2741014   

Inspecting layer: SAPOLYGON
Index(['AREASYMBOL', 'SPATIALVER', 'LKEY', 'Shape_Length', 'Shape_Area',
       'SOURCE', 'geometry'],
      dtype='object')
  AREASYMBOL  SPATIALVER   LKEY   Shape_Length    Shape_Area  SOURCE  \
0      CO636         3.0  14371  148538.744211  3.062635e+08  SSURGO   
1      KS145        11.0  10599  194571.262320  1.954684e+09  SSURGO   
2      CO099        13.0  10655  262716.959114  4.258098e+09  SSURGO   
3      CO011        12.0  10644  254960.334741  3.991455e+09  SSURGO   
4      CO089        11.0  10653  265676.565583  3.286150e+09  SSURGO   

                                            geometry  
0  MULTIPOLYGON (((-778418.1 1630725.1, -778446.3...  
1  MULTIPOLYGON (((-262470.1 1706056.6, -262483.9...  
2  MULTIPOLYGON (((-523488.3 1709604.5, -523532.5...  
3  MULTIPOLYGON (((-583830.5 1713440.2, -583877.8...  
4  MULTIPOLYGON (((-649477.6 1718195.3, -647923 1...   
```

### 'lkey' and 'mukey'  are in mapunit layer

In [34]:
mapunit_df = gpd.read_file(gdb_path, layer='mapunit')
mapunit_df

,musym,muname,mukind,mustatus,muacres,mapunitlfw_l,mapunitlfw_r,mapunitlfw_h,mapunitpfa_l,mapunitpfa_r,...,muwndhelcl,interpfocus,invesintens,iacornsr,nhiforsoigrp,nhspiagr,vtsepticsyscl,mucertstat,lkey,mukey
0,1030,"Udorthents-Pits, gravel complex",Complex,None,785.0,NaN,NaN,NaN,NaN,NaN,...,None,None,None,NaN,None,NaN,None,None,13785,398852
1,1033,Beaches-Menahga complex,Complex,None,598.0,NaN,NaN,NaN,NaN,NaN,...,None,None,None,NaN,None,NaN,None,None,13785,398853
2,1059,Wega silt loam,Consociation,None,2875.0,NaN,NaN,NaN,NaN,NaN,...,None,None,None,NaN,None,NaN,None,None,13785,398854
3,1066,"Rock outcrop-Garnes complex, very stony",Complex,None,733.0,NaN,NaN,NaN,NaN,NaN,...,None,None,None,NaN,None,NaN,None,None,13785,398855
4,1067,"Waupaca-Eutroboralfs complex, 0 to 60 percent ...",Association,None,8224.0,NaN,NaN,NaN,NaN,NaN,...,None,None,None,NaN,None,NaN,None,None,13785,398856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317725,308y0,"Ronneby-Giese, frequently ponded, complex, 0 t...",Complex,Correlated,5407.0,NaN,NaN,NaN,NaN,NaN,...,None,Wildlife Management,Order 2,NaN,None,NaN,None,"certified, all components",114876,3314511
317726,308xy,"Millward-Mora complex, 2 to 20 percent slopes,...",Complex,Correlated,577.0,NaN,NaN,NaN,NaN,NaN,...,None,Forest Management,Order 2,NaN,None,NaN,None,"certified, all components",114876,3314512
317727,308y8,"Milaca and Greysolon soils, 2 to 20 percent sl...",Undifferentiated group,Correlated,14930.0,NaN,NaN,NaN,NaN,NaN,...,None,Forest Management,Order 2,NaN,None,NaN,None,"certified, all components",114876,3314513
317728,308xm,"Milaca silt loam, 5 to 30 percent slopes, stony",Consociation,Correlated,108.0,NaN,NaN,NaN,NaN,NaN,...,None,Forest Management,Order 2,NaN,None,NaN,None,"certified, all components",114876,3314514


In [35]:
len(mapunit_df['lkey'].unique())

3220

In [36]:
mapunit_small = mapunit_df[['lkey', 'mukey']]
del mapunit_df
gc.collect()  # Force garbage collection

0

In [37]:
mapunit_small

,lkey,mukey
0,13785,398852
1,13785,398853
2,13785,398854
3,13785,398855
4,13785,398856
...,...,...
317725,114876,3314511
317726,114876,3314512
317727,114876,3314513
317728,114876,3314514


### soil boundaries are in 'SAPOLYGON' layer

In [38]:
# SAPOLYGON_gdf = gpd.read_file(gdb_path, layer='SAPOLYGON')
# SAPOLYGON_gdf

In [39]:
# len(SAPOLYGON_gdf['LKEY'].unique())

In [40]:
# # Get the unique values from both columns
# sapolygon_lkey_unique = set(SAPOLYGON_gdf['LKEY'].unique())
# mapunit_lkey_unique = set(mapunit_small['lkey'].unique())

# # Find common keys
# common_keys = sapolygon_lkey_unique.intersection(mapunit_lkey_unique)
# print(f"Number of common keys: {len(common_keys)}")

# # Find keys in SAPOLYGON_df but not in mapunit_df
# sapolygon_not_in_mapunit = sapolygon_lkey_unique.difference(mapunit_lkey_unique)
# print(f"Number of keys in SAPOLYGON but not in mapunit: {len(sapolygon_not_in_mapunit)}")

# # Find keys in mapunit_df but not in SAPOLYGON_df
# mapunit_not_in_sapolygon = mapunit_lkey_unique.difference(sapolygon_lkey_unique)
# print(f"Number of keys in mapunit but not in SAPOLYGON: {len(mapunit_not_in_sapolygon)}")

# # Optionally, print some of these keys for inspection
# print("Some keys in SAPOLYGON but not in mapunit:", list(sapolygon_not_in_mapunit)[:10])
# print("Some keys in mapunit but not in SAPOLYGON:", list(mapunit_not_in_sapolygon)[:10])

In [41]:
ph_small.columns

Index(['ph1to1h2o_l', 'ph1to1h2o_r', 'ph1to1h2o_h', 'ph01mcacl2_l',
       'ph01mcacl2_r', 'ph01mcacl2_h', 'cokey', 'chkey'],
      dtype='object')

In [42]:
mapunit_small.columns

Index(['lkey', 'mukey'], dtype='object')

### 'mukey' and 'cokey' are in 'component' layer

In [43]:
# component_df = gpd.read_file(gdb_path, layer='component')
# component_df.to_pickle('../datasets/soil/component_df.pickle')

component_df = gpd.GeoDataFrame(pd.read_pickle('../datasets/soil/component_df.pickle'))

In [44]:
component_df

,comppct_l,comppct_r,comppct_h,compname,compkind,majcompflag,otherph,localphase,slope_l,slope_r,...,flsoilleachpot,flsoirunoffpot,fltemik2use,fltriumph2use,indraingrp,innitrateleachi,misoimgmtgrp,vasoimgtgrp,mukey,cokey
0,NaN,60.0,NaN,Udorthents,Taxon above family,Yes,None,None,2.0,26.0,...,None,None,N/A,N/A,None,NaN,None,None,398852,24629854
1,NaN,40.0,NaN,Pits,Miscellaneous area,Yes,None,gravel,2.0,26.0,...,None,None,N/A,N/A,None,NaN,None,None,398852,24629855
2,NaN,30.0,NaN,Menahga,Series,Yes,LOW PPT,None,1.0,9.0,...,None,None,N/A,N/A,None,NaN,None,None,398853,24629637
3,NaN,60.0,NaN,Beaches,Miscellaneous area,Yes,None,None,NaN,NaN,...,None,None,N/A,N/A,None,NaN,None,None,398853,24629638
4,NaN,10.0,NaN,Very poorly drained soils,None,No,None,None,NaN,NaN,...,None,None,N/A,N/A,None,NaN,None,None,398853,24629639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131657,0.0,2.0,5.0,Scott Lake,Taxadjunct,No,Exhibit A,stony,5.0,7.0,...,None,None,3,3,None,NaN,None,None,3314514,3314514:2808079
1131658,85.0,93.0,100.0,Milaca,Series,Yes,Exhibit A,stony,5.0,20.0,...,None,None,3,3,None,NaN,None,None,3314514,3314514:2808146
1131659,0.0,10.0,15.0,Cathro,Series,No,Exhibit A,frequently ponded,0.0,0.0,...,None,None,3,3,None,NaN,None,None,3314515,3314515:2808110
1131660,0.0,60.0,100.0,Loxley,Series,Yes,Exhibit A,None,0.0,0.0,...,None,None,3,3,None,NaN,None,None,3314515,3314515:2808111


In [45]:
list(component_df.columns)

['comppct_l',
 'comppct_r',
 'comppct_h',
 'compname',
 'compkind',
 'majcompflag',
 'otherph',
 'localphase',
 'slope_l',
 'slope_r',
 'slope_h',
 'slopelenusle_l',
 'slopelenusle_r',
 'slopelenusle_h',
 'runoff',
 'tfact',
 'wei',
 'weg',
 'erocl',
 'earthcovkind1',
 'earthcovkind2',
 'hydricon',
 'hydricrating',
 'drainagecl',
 'elev_l',
 'elev_r',
 'elev_h',
 'aspectccwise',
 'aspectrep',
 'aspectcwise',
 'geomdesc',
 'albedodry_l',
 'albedodry_r',
 'albedodry_h',
 'airtempa_l',
 'airtempa_r',
 'airtempa_h',
 'map_l',
 'map_r',
 'map_h',
 'reannualprecip_l',
 'reannualprecip_r',
 'reannualprecip_h',
 'ffd_l',
 'ffd_r',
 'ffd_h',
 'nirrcapcl',
 'nirrcapscl',
 'nirrcapunit',
 'irrcapcl',
 'irrcapscl',
 'irrcapunit',
 'cropprodindex',
 'constreeshrubgrp',
 'wndbrksuitgrp',
 'rsprod_l',
 'rsprod_r',
 'rsprod_h',
 'foragesuitgrpid',
 'wlgrain',
 'wlgrass',
 'wlherbaceous',
 'wlshrub',
 'wlconiferous',
 'wlhardwood',
 'wlwetplant',
 'wlshallowwat',
 'wlrangeland',
 'wlopenland',
 'wlwood

In [46]:
component_columns_to_keep = [
    # 'comppct_l',
    # 'comppct_r',
    # 'comppct_h',
    # 'compname',
    # 'compkind',
    # 'majcompflag',
    # 'otherph',
    # 'localphase',
    # 'slope_l',
    # 'slope_r',
    # 'slope_h',
    # 'slopelenusle_l',
    # 'slopelenusle_r',
    # 'slopelenusle_h',
    # 'runoff',
    # 'tfact',
    # 'wei',
    # 'weg',
    # 'erocl',
    # 'earthcovkind1',
    # 'earthcovkind2',
    # 'hydricon',
    # 'hydricrating',
    # 'drainagecl',
    # 'elev_l',
    # 'elev_r',
    # 'elev_h',
    # 'aspectccwise',
    # 'aspectrep',
    # 'aspectcwise',
    # 'geomdesc',
    # 'albedodry_l',
    # 'albedodry_r',
    # 'albedodry_h',
    # 'airtempa_l',
    # 'airtempa_r',
    # 'airtempa_h',
    # 'map_l',
    # 'map_r',
    # 'map_h',
    # 'reannualprecip_l',
    # 'reannualprecip_r',
    # 'reannualprecip_h',
    # 'ffd_l',
    # 'ffd_r',
    # 'ffd_h',
    # 'nirrcapcl',
    # 'nirrcapscl',
    # 'nirrcapunit',
    # 'irrcapcl',
    # 'irrcapscl',
    # 'irrcapunit',
    # 'cropprodindex',
    # 'constreeshrubgrp',
    # 'wndbrksuitgrp',
    # 'rsprod_l',
    # 'rsprod_r',
    # 'rsprod_h',
    # 'foragesuitgrpid',
    # 'wlgrain',
    # 'wlgrass',
    # 'wlherbaceous',
    # 'wlshrub',
    # 'wlconiferous',
    # 'wlhardwood',
    # 'wlwetplant',
    # 'wlshallowwat',
    # 'wlrangeland',
    # 'wlopenland',
    # 'wlwoodland',
    # 'wlwetland',
    # 'soilslippot',
    # 'frostact',
    # 'initsub_l',
    # 'initsub_r',
    # 'initsub_h',
    # 'totalsub_l',
    # 'totalsub_r',
    # 'totalsub_h',
    # 'hydgrp',
    # 'corcon',
    # 'corsteel',
    # 'taxclname',
    # 'taxorder',
    # 'taxsuborder',
    # 'taxgrtgroup',
    # 'taxsubgrp',
    # 'taxpartsize',
    # 'taxpartsizemod',
    # 'taxceactcl',
    # 'taxreaction',
    # 'taxtempcl',
    # 'taxmoistscl',
    # 'taxtempregime',
    # 'soiltaxedition',
    # 'castorieindex',
    # 'flecolcomnum',
    # 'flhe',
    # 'flphe',
    # 'flsoilleachpot',
    # 'flsoirunoffpot',
    # 'fltemik2use',
    # 'fltriumph2use',
    # 'indraingrp',
    # 'innitrateleachi',
    # 'misoimgmtgrp',
    # 'vasoimgtgrp',
    'mukey',
    'cokey'
]

In [47]:
# component_small = component_df[['mukey','cokey']]
component_small = component_df[component_columns_to_keep]
del component_df
gc.collect()

0

## check key columns

In [48]:
# mapunit_small = mapunit_df[['lkey', 'mukey']]
mapunit_small

,lkey,mukey
0,13785,398852
1,13785,398853
2,13785,398854
3,13785,398855
4,13785,398856
...,...,...
317725,114876,3314511
317726,114876,3314512
317727,114876,3314513
317728,114876,3314514


In [49]:
# ph_small = gdf_soil[pH_columns_to_keep]
ph_small[['cokey', 'chkey']]

,cokey,chkey
0,24629637,73203533
1,24629637,73203532
2,24630059,73204462
3,24630059,73204461
4,24630061,73204464
...,...,...
3563603,3314515:2808111,3314515:11333113
3563604,3314515:2808111,3314515:11333112
3563605,3314515:2808112,3314515:11333115
3563606,3314515:2808112,3314515:11333116


In [50]:
component_small[['mukey', 'cokey']]

,mukey,cokey
0,398852,24629854
1,398852,24629855
2,398853,24629637
3,398853,24629638
4,398853,24629639
...,...,...
1131657,3314514,3314514:2808079
1131658,3314514,3314514:2808146
1131659,3314515,3314515:2808110
1131660,3314515,3314515:2808111


In [51]:
fields_joined_with_soils[['LKEY']]

,LKEY
478801,10644
107446,10316
121162,10306
88813,10323
516266,10650
...,...
119071,18327
540278,10660
591949,10263
217842,10657


In [52]:
fields_joined_with_soils.columns

Index(['CSBID', 'CSBYEARS', 'CSBACRES', 'CDL2016', 'CDL2017', 'CDL2018',
       'CDL2019', 'CDL2020', 'CDL2021', 'CDL2022', 'CDL2023', 'STATEFIPS',
       'STATEASD', 'ASD', 'CNTY', 'CNTYFIPS', 'INSIDE_X', 'INSIDE_Y',
       'Shp_Len', 'Shp_Area', 'geometry', 'Longitude', 'Latitude', 'Elevation',
       'color', 'Crop', 'index_SAPOLYGON', 'LKEY'],
      dtype='object')

## perform merges

In [53]:
# merge_gdf = SAPOLYGON_gdf.merge(mapunit_small, left_on='LKEY', right_on='lkey', how='left')
merge_gdf = fields_joined_with_soils.merge(mapunit_small, left_on='LKEY', right_on='lkey', how='left')
del mapunit_small
gc.collect()

0

In [54]:
merge_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 176226 entries, 0 to 176225
Data columns (total 30 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   CSBID            176226 non-null  object  
 1   CSBYEARS         176226 non-null  object  
 2   CSBACRES         176226 non-null  float64 
 3   CDL2016          176226 non-null  int64   
 4   CDL2017          176226 non-null  int64   
 5   CDL2018          176226 non-null  int64   
 6   CDL2019          176226 non-null  int64   
 7   CDL2020          176226 non-null  int64   
 8   CDL2021          176226 non-null  int64   
 9   CDL2022          176226 non-null  int64   
 10  CDL2023          176226 non-null  int64   
 11  STATEFIPS        176226 non-null  object  
 12  STATEASD         176226 non-null  object  
 13  ASD              176226 non-null  object  
 14  CNTY             176226 non-null  object  
 15  CNTYFIPS         176226 non-null  object  
 16  INSIDE_X    

In [55]:
merge_gdf = merge_gdf.merge(component_small, on='mukey', how='left')
del component_small
gc.collect()

0

In [56]:
merge_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 891764 entries, 0 to 891763
Data columns (total 31 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   CSBID            891764 non-null  object  
 1   CSBYEARS         891764 non-null  object  
 2   CSBACRES         891764 non-null  float64 
 3   CDL2016          891764 non-null  int64   
 4   CDL2017          891764 non-null  int64   
 5   CDL2018          891764 non-null  int64   
 6   CDL2019          891764 non-null  int64   
 7   CDL2020          891764 non-null  int64   
 8   CDL2021          891764 non-null  int64   
 9   CDL2022          891764 non-null  int64   
 10  CDL2023          891764 non-null  int64   
 11  STATEFIPS        891764 non-null  object  
 12  STATEASD         891764 non-null  object  
 13  ASD              891764 non-null  object  
 14  CNTY             891764 non-null  object  
 15  CNTYFIPS         891764 non-null  object  
 16  INSIDE_X    

In [57]:
merge_gdf = merge_gdf.merge(ph_small, on='cokey', how='left')
del ph_small
gc.collect()

0

In [58]:
merge_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2635249 entries, 0 to 2635248
Data columns (total 38 columns):
 #   Column           Dtype   
---  ------           -----   
 0   CSBID            object  
 1   CSBYEARS         object  
 2   CSBACRES         float64 
 3   CDL2016          int64   
 4   CDL2017          int64   
 5   CDL2018          int64   
 6   CDL2019          int64   
 7   CDL2020          int64   
 8   CDL2021          int64   
 9   CDL2022          int64   
 10  CDL2023          int64   
 11  STATEFIPS        object  
 12  STATEASD         object  
 13  ASD              object  
 14  CNTY             object  
 15  CNTYFIPS         object  
 16  INSIDE_X         float64 
 17  INSIDE_Y         float64 
 18  Shp_Len          float64 
 19  Shp_Area         float64 
 20  geometry         geometry
 21  Longitude        float64 
 22  Latitude         float64 
 23  Elevation        int32   
 24  color            object  
 25  Crop             object  
 26  index_

In [59]:
len(merge_gdf)

2635249

In [60]:
# merge_gdf.to_parquet('../datasets/soil/field_soil_ph_merge_gdf.parquet')
merge_gdf.to_parquet('../data/soil/field_soil_ph_merge_gdf.parquet')

In [61]:
merge_gdf

,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,lkey,mukey,cokey,ph1to1h2o_l,ph1to1h2o_r,ph1to1h2o_h,ph01mcacl2_l,ph01mcacl2_r,ph01mcacl2_h,chkey
0,081623012787392,1623,3.791540,4,24,61,61,24,24,61,...,10644,94229,23646547,7.9,8.0,8.4,7.4,7.8,8.4,70181879
1,081623012787392,1623,3.791540,4,24,61,61,24,24,61,...,10644,94229,23646547,7.9,8.6,9.0,7.4,8.2,8.4,70181878
2,081623012787392,1623,3.791540,4,24,61,61,24,24,61,...,10644,94229,23646547,7.9,8.6,9.0,7.4,8.2,8.4,70181876
3,081623012787392,1623,3.791540,4,24,61,61,24,24,61,...,10644,94229,23646547,7.9,8.6,9.0,7.4,8.2,8.4,70181877
4,081623012787392,1623,3.791540,4,24,61,61,24,24,61,...,10644,94229,23646548,7.4,8.2,8.5,6.9,7.8,8.0,70181880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635244,081623008142809,1623,2.358687,24,24,24,176,176,176,176,...,10649,3223698,23656210,7.4,8.1,8.4,6.6,7.6,7.9,70213901
2635245,081623008142809,1623,2.358687,24,24,24,176,176,176,176,...,10649,3223698,23656211,NaN,NaN,NaN,NaN,NaN,NaN,70213904
2635246,081623008142809,1623,2.358687,24,24,24,176,176,176,176,...,10649,3223698,23656212,6.1,7.0,7.8,5.5,6.5,7.3,70213905
2635247,081623008142809,1623,2.358687,24,24,24,176,176,176,176,...,10649,3223698,23656212,6.1,7.0,7.8,5.5,6.5,7.3,70213907


In [62]:
merge_gdf.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [63]:
list(merge_gdf.columns)

['CSBID',
 'CSBYEARS',
 'CSBACRES',
 'CDL2016',
 'CDL2017',
 'CDL2018',
 'CDL2019',
 'CDL2020',
 'CDL2021',
 'CDL2022',
 'CDL2023',
 'STATEFIPS',
 'STATEASD',
 'ASD',
 'CNTY',
 'CNTYFIPS',
 'INSIDE_X',
 'INSIDE_Y',
 'Shp_Len',
 'Shp_Area',
 'geometry',
 'Longitude',
 'Latitude',
 'Elevation',
 'color',
 'Crop',
 'index_SAPOLYGON',
 'LKEY',
 'lkey',
 'mukey',
 'cokey',
 'ph1to1h2o_l',
 'ph1to1h2o_r',
 'ph1to1h2o_h',
 'ph01mcacl2_l',
 'ph01mcacl2_r',
 'ph01mcacl2_h',
 'chkey']

## groupby fields

In [64]:
# groupby_fields = merge_gdf.groupby('CSBID')

In [65]:
# for i, df in enumerate(groupby_fields):
#     print()
#     print('****')
#     display(df)
#     if i > 0:
#         break

In [66]:
# Assuming merge_df is the dataframe containing your data
groupby_fields = merge_gdf.groupby('CSBID').agg({
    'CSBYEARS': 'first',
    'CSBACRES': 'first',
    'CDL2016': 'first',
    'CDL2017': 'first',
    'CDL2018': 'first',
    'CDL2019': 'first',
    'CDL2020': 'first',
    'CDL2021': 'first',
    'CDL2022': 'first',
    'CDL2023': 'first',
    'STATEFIPS': 'first',
    'STATEASD': 'first',
    'ASD': 'first',
    'CNTY': 'first',
    'CNTYFIPS': 'first',
    'INSIDE_X': 'first',
    'INSIDE_Y': 'first',
    'Shp_Len': 'first',
    'Shp_Area': 'first',
    'geometry': 'first',
    'Longitude': 'first',
    'Latitude': 'first',
    'Elevation': 'first',
    'color': 'first',
    'Crop': 'first',
    'ph1to1h2o_r': ['mean', 'min', 'max', 'std']
})

# Flatten the MultiIndex in columns
groupby_fields.columns = ['_'.join(col).strip() for col in groupby_fields.columns.values]
groupby_fields.reset_index(inplace=True)

In [67]:
# Display the resulting dataframe
groupby_fields

,CSBID,CSBYEARS_first,CSBACRES_first,CDL2016_first,CDL2017_first,CDL2018_first,CDL2019_first,CDL2020_first,CDL2021_first,CDL2022_first,...,geometry_first,Longitude_first,Latitude_first,Elevation_first,color_first,Crop_first,ph1to1h2o_r_mean,ph1to1h2o_r_min,ph1to1h2o_r_max,ph1to1h2o_r_std
0,041623001737287,1623,2.882065,61,61,74,74,152,61,61,...,"MULTIPOLYGON (((-1234020.598 1157021.234, -123...",-109.282459,32.706889,1254,#b6705c,Pecans,8.018182,6.1,10.1,0.845030
1,041623001737315,1623,2.708718,61,61,74,152,152,152,152,...,"MULTIPOLYGON (((-1254751.575 1148858.857, -125...",-109.489526,32.609445,1021,#c7d79e,Shrubland,8.018182,6.1,10.1,0.845030
2,041623001737326,1623,2.952763,2,74,74,61,61,152,61,...,"MULTIPOLYGON (((-1259531.153 1130972.628, -125...",-109.514170,32.443414,1088,#bfbf7a,Fallow/Idle Cropland,8.018182,6.1,10.1,0.845030
3,041623001737406,1623,12.094357,61,61,61,152,152,152,152,...,"MULTIPOLYGON (((-1255741.302 1119160.528, -125...",-109.454658,32.344597,1128,#b6705c,Pecans,8.018182,6.1,10.1,0.845030
4,041623004560244,1623,5.451031,61,61,42,51,36,22,47,...,"MULTIPOLYGON (((-1563229.963 1346827.806, -156...",-113.123487,33.915456,673,#ff6666,Misc Vegs & Fruits,8.083302,7.5,8.5,0.232498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,491623016053358,1623,3.711081,152,152,152,152,24,61,152,...,"MULTIPOLYGON (((-1212362.959 1843540.588, -121...",-110.144871,38.805319,1255,#c7d79e,Shrubland,7.107234,4.0,10.1,1.103962
1196,491623016053395,1623,5.684277,152,36,36,36,36,36,36,...,"MULTIPOLYGON (((-1206060.153 1839381.109, -120...",-110.064416,38.776194,1252,#ffa8e3,Alfalfa,8.264129,7.2,9.4,0.432650
1197,491623016053447,1623,33.585986,36,36,36,36,36,36,36,...,"MULTIPOLYGON (((-1265716.79 1805871.256, -1265...",-110.689771,38.399610,1341,#ffa8e3,Alfalfa,8.224951,6.1,10.1,0.402092
1198,491623016053449,1623,2.936797,152,152,1,1,1,36,152,...,"MULTIPOLYGON (((-1157384.118 1768393.098, -115...",-109.389405,38.212378,1763,#c7d79e,Shrubland,7.756326,5.0,9.0,0.699584


In [68]:
# list(groupby_fields.columns)

In [69]:
groupby_fields_column_names = [
    'CSBID',
    'CSBYEARS',
    'CSBACRES',
    'CDL2016',
    'CDL2017',
    'CDL2018',
    'CDL2019',
    'CDL2020',
    'CDL2021',
    'CDL2022',
    'CDL2023',
    'STATEFIPS',
    'STATEASD',
    'ASD',
    'CNTY',
    'CNTYFIPS',
    'INSIDE_X',
    'INSIDE_Y',
    'Shp_Len',
    'Shp_Area',
    'geometry',
    'Longitude',
    'Latitude',
    'Elevation',
    'color',
    'Crop',
    'ph1to1h2o_r_mean',
    'ph1to1h2o_r_min',
    'ph1to1h2o_r_max',
    'ph1to1h2o_r_std'
]

In [70]:
groupby_fields.columns = groupby_fields_column_names

In [71]:
groupby_fields = groupby_fields.set_geometry('geometry')

In [72]:
groupby_fields.set_crs(merge_gdf.crs, inplace=True)

,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,geometry,Longitude,Latitude,Elevation,color,Crop,ph1to1h2o_r_mean,ph1to1h2o_r_min,ph1to1h2o_r_max,ph1to1h2o_r_std
0,041623001737287,1623,2.882065,61,61,74,74,152,61,61,...,"MULTIPOLYGON (((-1234020.598 1157021.234, -123...",-109.282459,32.706889,1254,#b6705c,Pecans,8.018182,6.1,10.1,0.845030
1,041623001737315,1623,2.708718,61,61,74,152,152,152,152,...,"MULTIPOLYGON (((-1254751.575 1148858.857, -125...",-109.489526,32.609445,1021,#c7d79e,Shrubland,8.018182,6.1,10.1,0.845030
2,041623001737326,1623,2.952763,2,74,74,61,61,152,61,...,"MULTIPOLYGON (((-1259531.153 1130972.628, -125...",-109.514170,32.443414,1088,#bfbf7a,Fallow/Idle Cropland,8.018182,6.1,10.1,0.845030
3,041623001737406,1623,12.094357,61,61,61,152,152,152,152,...,"MULTIPOLYGON (((-1255741.302 1119160.528, -125...",-109.454658,32.344597,1128,#b6705c,Pecans,8.018182,6.1,10.1,0.845030
4,041623004560244,1623,5.451031,61,61,42,51,36,22,47,...,"MULTIPOLYGON (((-1563229.963 1346827.806, -156...",-113.123487,33.915456,673,#ff6666,Misc Vegs & Fruits,8.083302,7.5,8.5,0.232498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,491623016053358,1623,3.711081,152,152,152,152,24,61,152,...,"MULTIPOLYGON (((-1212362.959 1843540.588, -121...",-110.144871,38.805319,1255,#c7d79e,Shrubland,7.107234,4.0,10.1,1.103962
1196,491623016053395,1623,5.684277,152,36,36,36,36,36,36,...,"MULTIPOLYGON (((-1206060.153 1839381.109, -120...",-110.064416,38.776194,1252,#ffa8e3,Alfalfa,8.264129,7.2,9.4,0.432650
1197,491623016053447,1623,33.585986,36,36,36,36,36,36,36,...,"MULTIPOLYGON (((-1265716.79 1805871.256, -1265...",-110.689771,38.399610,1341,#ffa8e3,Alfalfa,8.224951,6.1,10.1,0.402092
1198,491623016053449,1623,2.936797,152,152,1,1,1,36,152,...,"MULTIPOLYGON (((-1157384.118 1768393.098, -115...",-109.389405,38.212378,1763,#c7d79e,Shrubland,7.756326,5.0,9.0,0.699584


In [73]:
del merge_gdf
gc.collect()

0

## save field dataframe with mean, min, max, std pH columns

In [74]:
# groupby_fields.to_parquet('../datasets/soil/field_soil_ph_mean_gdf.parquet')
groupby_fields.to_parquet('../data/fields/csb_sample_with_soil_ph.parquet')

## calculate pH scores

### set fields df name

In [75]:
fields = groupby_fields

### load ecocrop data and set columns

Imputed, includes USDA Hardiness Zones

In [76]:
ecocrop_file = '../data/crops/EcoCrop_Clean_Imputed_All.pickle'
ecocrop_all = pd.read_pickle(ecocrop_file)
ecocrop_all

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Crop_Cycle_Max,Use_Main,Use_Detailed,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL
0,289,Abelmoschus esculentus,Abelmoschus,esculentus,NaN,herb,erect,annual,single stem,vegetables,...,180.0,food & beverage,vitamins,fruits,"tropical wet & dry (Aw), tropical wet (Ar), st...",5-11,5,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abe...
1,290,Abelmoschus manihot,Abelmoschus,manihot,NaN,shrub,erect,"annual, perennial","deciduous, multi stem","vegetables, ornamentals/turf, medicinals & aro...",...,365.0,food & beverage,vitamins,leaves,"tropical wet & dry (Aw), tropical wet (Ar)",8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abe...
2,291,Abelmoschus moschatus,Abelmoschus,moschatus,NaN,"herb, sub-shrub",prostrate/procumbent/semi-erect,"annual, biennial, perennial","deciduous, multi stem","ornamentals/turf, medicinals & aromatic",...,0.0,environmental,ornamental/turf,entire plant,tropical wet & dry (Aw),8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abe...
3,295,Acacia auriculiformis,Acacia,auriculiformis,NaN,tree,erect,perennial,single stem,forest/wood,...,240.0,material,dye/tannin,stems,"tropical wet & dry (Aw), tropical wet (Ar)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...
4,297,Acacia farnesiana,Acacia,farnesiana,NaN,tree,erect,perennial,single stem,"materials, ornamentals/turf, medicinals & arom...",...,240.0,environmental,ornamental/turf,entire plant,"tropical wet & dry (Aw), steppe or semiarid (B...",9-11,9,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563,400000,Chamaecrista rotundifolia,Chamaecrista,rotundifolia,NaN,herb,prostrate/procumbent/semi-erect,"annual, perennial","deciduous, multi stem","forage/pasture, medicinals & aromatic",...,0.0,animal food (feed),vitamins,entire plant,"tropical wet & dry (Aw), tropical wet (Ar), su...",9-11,9,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Cha...
2564,400001,Acacia polyacantha,Acacia,polyacantha,NaN,"shrub, tree",erect,perennial,"single stem, multi stem","forest/wood, environmental",...,0.0,fuels,fuelwood,bark,"desert or arid (Bw), steppe or semiarid (Bs)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...
2565,400002,Prosopis affinis,Prosopis,affinis,NaN,tree,erect,perennial,"deciduous, single stem, C3 photosynthesis",forest/wood,...,0.0,food & beverage,vitamins,bark,"tropical wet & dry (Aw), desert or arid (Bw), ...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Pro...
2566,400003,Vicia dasycarpa,Vicia,dasycarpa,NaN,herb,climber/scrambler/scadent,annual,multi stem,forage/pasture,...,0.0,animal food (feed),minerals,entire plant,"tropical wet & dry (Aw), tropical wet (Ar), st...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Vic...


In [77]:
# list(ecocrop_all.columns)

In [78]:
crop_columns = [
    'Crop_Code',
    'Scientific_Name',
    'Genus',
    'Species',
    # 'Variety',
    # 'Life_Form',
    # 'Habit',
    # 'Life_Span',
    # 'Physiology',
    # 'Category',
    # 'Plant_Attributes',
    # 'Temp_Opt_Min',
    # 'Temp_Opt_Max',
    # 'Temp_Opt_Min_F',
    # 'Temp_Opt_Max_F',
    # 'Temp_Abs_Min',
    # 'Temp_Abs_Max',
    # 'Temp_Abs_Min_F',
    # 'Temp_Abs_Max_F',
    # 'Killing_Temp_Rest',
    # 'Killing_Temp_Growth',
    # 'Killing_Temp_Rest_F',
    # 'Killing_Temp_Growth_F',
    # 'Rain_Opt_Min',
    # 'Rain_Opt_Max',
    # 'Rain_Abs_Min',
    # 'Rain_Abs_Max',
    # 'Lat_Opt_Min',
    # 'Lat_Opt_Max',
    # 'Lat_Abs_Min',
    # 'Lat_Abs_Max',
    # 'Alt_Opt_Min',
    # 'Alt_Opt_Max',
    # 'Alt_Abs_Min',
    # 'Alt_Abs_Max',
    'pH_Opt_Min',
    'pH_Opt_Max',
    'pH_Abs_Min',
    'pH_Abs_Max',
    # 'Light_Opt_Min',
    # 'Light_Opt_Max',
    # 'Light_Abs_Min',
    # 'Light_Abs_Max',
    # 'Depth_Opt',
    # 'Depth_Abs',
    # 'Texture_Ops',
    # 'Texture_Abs',
    # 'Fertility_Ops',
    # 'Fertility_Abs',
    # 'Al_Toxicity_Opt',
    # 'Al_Toxicity_Abs',
    # 'Salinity_Ops',
    # 'Salinity_Abs',
    # 'Drainage_Opt',
    # 'Drainage_Abs',
    # 'Photoperiod',
    # 'Abiotic_Tolererance',
    # 'Abiotic_Susceptibility',
    # 'Introduction_Risks',
    # 'Production_System',
    # 'Cropping_System',
    # 'Subsystem',
    # 'Companion_Species',
    # 'Level_of_Mechanization',
    # 'Labour_Intensity',
    # 'Crop_Cycle_Min',
    # 'Crop_Cycle_Max',
    # 'Use_Main',
    # 'Use_Detailed',
    # 'Use_Part',
    # 'Climate_Zone_Trewartha',
    # 'USDA_Hardiness_Zone',
    # 'USDA_Hardiness_Zone_Min',
    # 'USDA_Hardiness_Zone_Max',
    # 'Datasheet_URL',
    # 'PFAF_URL'
]

In [79]:
crops = ecocrop_all[crop_columns]

In [80]:
crops

,Crop_Code,Scientific_Name,Genus,Species,pH_Opt_Min,pH_Opt_Max,pH_Abs_Min,pH_Abs_Max
0,289,Abelmoschus esculentus,Abelmoschus,esculentus,5.500000,7.000000,4.500000,8.700000
1,290,Abelmoschus manihot,Abelmoschus,manihot,5.500000,7.000000,5.000000,8.000000
2,291,Abelmoschus moschatus,Abelmoschus,moschatus,5.500000,7.000000,5.000000,8.000000
3,295,Acacia auriculiformis,Acacia,auriculiformis,6.000000,8.000000,3.000000,9.500000
4,297,Acacia farnesiana,Acacia,farnesiana,6.500000,7.000000,4.300000,8.000000
...,...,...,...,...,...,...,...,...
2563,400000,Chamaecrista rotundifolia,Chamaecrista,rotundifolia,5.000000,6.500000,4.500000,7.000000
2564,400001,Acacia polyacantha,Acacia,polyacantha,5.766304,7.021739,5.041304,7.698913
2565,400002,Prosopis affinis,Prosopis,affinis,5.900000,7.050000,5.300000,7.770000
2566,400003,Vicia dasycarpa,Vicia,dasycarpa,6.227273,7.090909,5.063636,8.072727


In [81]:
crops.columns

Index(['Crop_Code', 'Scientific_Name', 'Genus', 'Species', 'pH_Opt_Min',
       'pH_Opt_Max', 'pH_Abs_Min', 'pH_Abs_Max'],
      dtype='object')

### calculate field x crop pH score matrix 

In [82]:
fields.columns

Index(['CSBID', 'CSBYEARS', 'CSBACRES', 'CDL2016', 'CDL2017', 'CDL2018',
       'CDL2019', 'CDL2020', 'CDL2021', 'CDL2022', 'CDL2023', 'STATEFIPS',
       'STATEASD', 'ASD', 'CNTY', 'CNTYFIPS', 'INSIDE_X', 'INSIDE_Y',
       'Shp_Len', 'Shp_Area', 'geometry', 'Longitude', 'Latitude', 'Elevation',
       'color', 'Crop', 'ph1to1h2o_r_mean', 'ph1to1h2o_r_min',
       'ph1to1h2o_r_max', 'ph1to1h2o_r_std'],
      dtype='object')

In [83]:
# Define the vectorized score calculation function
# Define the vectorized score calculation function
def calculate_ph_scores(fields, crops):
    # Initialize an empty numpy array to store the scores
    scores = np.zeros((fields.shape[0], crops.shape[0]))

    # Extract the necessary columns as numpy arrays
    field_avg = fields['ph1to1h2o_r_mean'].values[:, np.newaxis]

    optimal_min = crops['pH_Opt_Min'].values[np.newaxis, :]
    optimal_max = crops['pH_Opt_Max'].values[np.newaxis, :]
    absolute_min = crops['pH_Abs_Min'].values[np.newaxis, :]
    absolute_max = crops['pH_Abs_Max'].values[np.newaxis, :]

    # Apply the scoring logic
    # +1 if the field mean pH is within the optimal range
    in_optimal_range = (optimal_min <= field_avg) & (field_avg <= optimal_max)
    scores[in_optimal_range] = 1.0

    # 0 if the field mean pH is within the absolute range but not in the optimal range
    in_absolute_range = (absolute_min <= field_avg) & (field_avg <= absolute_max)
    scores[in_absolute_range & ~in_optimal_range] = 0.0

    # -1 if the field mean pH is outside the absolute range
    scores[~in_absolute_range] = -1.0

    return scores

# Apply the vectorized function to calculate the score matrix
score_matrix = calculate_ph_scores(fields, crops)

# Apply the vectorized function to calculate the score matrix
score_matrix = calculate_ph_scores(fields, crops)

In [84]:
# Convert the numpy array back to a DataFrame with appropriate row and column labels
score_df = pd.DataFrame(score_matrix, index=fields['CSBID'], columns=crops['Scientific_Name'])
score_df

Scientific_Name,Abelmoschus esculentus,Abelmoschus manihot,Abelmoschus moschatus,Acacia auriculiformis,Acacia farnesiana,Acacia holosericea,Acacia mellifera,Adenanthera pavonina,Aeschynomene americana,Agathis australis,...,Teramnus micans,Thymus syriacus,Thysanolaena latifolia,Triodia pungens,Zornia orbiculata,Chamaecrista rotundifolia,Acacia polyacantha,Prosopis affinis,Vicia dasycarpa,Camelina sativa
CSBID,,,,,,,,,,,,,,,,,,,,,
041623001737287,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0
041623001737315,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0
041623001737326,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0
041623001737406,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0
041623004560244,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491623016053358,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,-1.0,...,1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0
491623016053395,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
491623016053447,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [85]:
score_df.describe()

Scientific_Name,Abelmoschus esculentus,Abelmoschus manihot,Abelmoschus moschatus,Acacia auriculiformis,Acacia farnesiana,Acacia holosericea,Acacia mellifera,Adenanthera pavonina,Aeschynomene americana,Agathis australis,...,Teramnus micans,Thymus syriacus,Thysanolaena latifolia,Triodia pungens,Zornia orbiculata,Chamaecrista rotundifolia,Acacia polyacantha,Prosopis affinis,Vicia dasycarpa,Camelina sativa
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.00000,1200.000000,1200.000000,...,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,0.046667,-0.235833,-0.235833,0.709167,-0.248333,0.219167,-0.545833,-0.72500,-0.235833,-0.987500,...,-0.175833,-0.345833,-0.560833,-0.560833,-0.945000,-0.937500,-0.545833,-0.470833,-0.122500,-0.560833
std,0.226270,0.529575,0.529575,0.454336,0.511744,0.427729,0.590076,0.54738,0.529575,0.111149,...,0.604335,0.512951,0.543027,0.543027,0.249052,0.289246,0.590076,0.599541,0.468341,0.543027
min,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,-1.00000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000
50%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-1.000000,-1.00000,0.000000,-1.000000,...,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000
75%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-1.00000,0.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [86]:
# Define the scenarios
scenarios = ['current_1', 'current_2', 'current_3']

# Repeat the 2D matrix along the third dimension (s)
# np.newaxis is used to add a new dimension to the array
ph_scores = np.repeat(score_matrix[:, :, np.newaxis], len(scenarios), axis=2)

# Convert the results matrix to an xarray DataArray
results_da = xr.DataArray(
    ph_scores,
    dims=['fields', 'crops', 'scenarios'],
    coords={
        'fields': fields['CSBID'],
        'crops': crops['Scientific_Name'],
        'scenarios': scenarios,
    },
    name='crop_suitability_pH'
)

# Now `results_da` is a 3D DataArray where the third dimension contains duplicates of the 2D score matrix

In [87]:
# Save the DataArray to a NetCDF file
results_da.to_netcdf('../data/scores/score_matrix_pH.nc')

In [88]:
# fields['CSBID']

## analyze pH scores

In [89]:
def get_crops_with_score_1(scores_matrix, crop_data):
    # Boolean mask where scores are 1
    suitable_mask = scores_matrix == 1
    
    # Get the crop names for each crop
    crop_names = np.array(crop_data['Scientific_Name'])
    
    # Create a list of crops with score 1 for each field using a list comprehension
    suitable_crops = {
        field_idx: crop_names[suitable_mask[field_idx]].tolist()
        for field_idx in range(scores_matrix.shape[0])
    }
    
    return suitable_crops

In [90]:
# get_crops_with_score_1(score_matrix,crops)

In [91]:
# Initialize the DataFrame to store the results using CSBID from fields
crops_with_score_1_df = pd.DataFrame({'CSBID': fields.index.astype('int64')})

# Get the crops with a score of 1 from the ph_score_matrix
crops_with_score_1 = get_crops_with_score_1(score_matrix, crops)

# Convert the results to a DataFrame for easier visualization
scenario_column = 'Suitable_Crops_pH_current'
number_column = 'Number_of_Suitable_Crops_pH_current'
temp_df = pd.DataFrame(list(crops_with_score_1.items()), columns=['CSBID', scenario_column])
temp_df[number_column] = temp_df[scenario_column].apply(len)

# Merge with the main DataFrame
crops_with_score_1_df = crops_with_score_1_df.merge(temp_df, on='CSBID', how='outer')
crops_with_score_1_df['CSBID'] = fields['CSBID']
crops_with_score_1_df

,CSBID,Suitable_Crops_pH_current,Number_of_Suitable_Crops_pH_current
0,041623001737287,"[Pongamia pinnata, Tecoma stans, Cicer arietin...",22
1,041623001737315,"[Pongamia pinnata, Tecoma stans, Cicer arietin...",22
2,041623001737326,"[Pongamia pinnata, Tecoma stans, Cicer arietin...",22
3,041623001737406,"[Pongamia pinnata, Tecoma stans, Cicer arietin...",22
4,041623004560244,"[Pongamia pinnata, Tecoma stans, Cicer arietin...",22
...,...,...,...
1195,491623016053358,"[Acacia auriculiformis, Acacia holosericea, Ag...",635
1196,491623016053395,"[Pongamia pinnata, Tecoma stans, Cicer arietin...",21
1197,491623016053447,"[Pongamia pinnata, Tecoma stans, Cicer arietin...",21
1198,491623016053449,"[Acacia auriculiformis, Aleurites moluccana, A...",206


In [92]:
data_for_plot = crops_with_score_1_df[['CSBID', 'Number_of_Suitable_Crops_pH_current']]
data_for_plot
# data_for_plot.info()

,CSBID,Number_of_Suitable_Crops_pH_current
0,041623001737287,22
1,041623001737315,22
2,041623001737326,22
3,041623001737406,22
4,041623004560244,22
...,...,...
1195,491623016053358,635
1196,491623016053395,21
1197,491623016053447,21
1198,491623016053449,206


In [93]:
crop_suitability_columns = [
    'Number_of_Suitable_Crops_pH_current', 
    # 'Number_of_Suitable_Crops_pH_current', 
    # 'Number_of_Suitable_Crops_pH_current', 
]

for column in crop_suitability_columns:
    print(f'{column:>50}  {crops_with_score_1_df[column].min():>8.2f} (Min),  {crops_with_score_1_df[column].max():>8.1f} (Max),  {crops_with_score_1_df[column].mean():>8.1f} (Avg)')

               Number_of_Suitable_Crops_pH_current      3.00 (Min),    1746.0 (Max),     278.0 (Avg)


In [94]:
data_for_plot = crops_with_score_1_df[['Number_of_Suitable_Crops_pH_current']]#.dropna()
data_for_plot

,Number_of_Suitable_Crops_pH_current
0,22
1,22
2,22
3,22
4,22
...,...
1195,635
1196,21
1197,21
1198,206


In [95]:
# Set bin size
bin_size = 20  # You can adjust the bin size as needed
color='cornflowerblue'
opacity=0.6

# Create Altair histogram of suitable crops
chart_h = alt.Chart(data_for_plot).mark_bar(color=color, opacity=opacity).encode(
    x=alt.X('Number_of_Suitable_Crops_pH_current:Q', bin=alt.Bin(step=bin_size), title='Binned Number of Suitable Crops', axis=alt.Axis(offset=10)),
    y=alt.Y('count()', title='Number of Fields', axis=alt.Axis(offset=10)),
).properties(
    title='Current (2023)',
    width=600,
    height=200
)

chart_h

alt.Chart(...)